In [1]:
import warnings

from NeuralNetwork.CNN import CNN_MEL, CNN_CHR, CNN_MFCC

warnings.filterwarnings("ignore")

2024-01-03 00:52:57.818948: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-03 00:52:57.820175: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-03 00:52:57.838201: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-03 00:52:57.838221: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-03 00:52:57.838767: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
try:
    del model_mel
    del model_chr
    del model_mfcc
    print("clean!")
except:
    pass
model_mel = CNN_MEL((128, 1293, 1))
model_chr = CNN_CHR((12, 1293, 1))
model_mfcc = CNN_MFCC((20, 1293, 1))

2024-01-03 00:52:58.695131: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-03 00:52:58.695483: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-03 00:52:58.717939: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required l

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 1293, 32)     320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 646, 32)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 646, 64)       18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 32, 323, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 323, 128)      73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 16, 161, 128)      0

In [ ]:
model_mel.train("./dataset/mel_format", epoch=10)
model_chr.train("./dataset/chr_format", epoch=5)
model_mfcc.train("./dataset/mfcc_format", epoch=20)

In [ ]:
model_mel.save_model("./model/model_mel.h5")
model_chr.save_model("./model/model_chr.h5")
model_mfcc.save_model("./model/model_mfcc.h5")

In [3]:
model_mel.load_model("./model/model_mel.h5")
model_chr.load_model("./model/model_chr.h5")
model_mfcc.load_model("./model/model_mfcc.h5")

In [4]:
import numpy as np
import librosa

y, sr = librosa.load("./dataset/test_long/On Va S'aimer Encore.mp3")
# y, sr = librosa.load("./dataset/audio_format/metal/metal.00011.wav")

spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)

mel_pic = librosa.power_to_db(spectrogram, ref=np.max)

harmonic = librosa.effects.harmonic(y)  ##
harmonic_features = librosa.feature.chroma_cqt(y=harmonic, sr=sr)  ##

mfccs = librosa.feature.mfcc(y=y, sr=sr)

duration = len(y) / sr

ans_mel, pre_mel = model_mel.predict(mel_pic, duration)
print(pre_mel)
print(ans_mel)
ans_chr, pre_chr = model_chr.predict(harmonic_features, duration)
print(pre_chr)
print(ans_chr)
ans_mfcc, pre_mfcc = model_mfcc.predict(mfccs, duration)
print(pre_mfcc)
print(ans_mfcc)
class_index = np.argmax((pre_mfcc * 0.5 + 2 * pre_chr + pre_mel * 0.5) / 3, axis=1)[0]
print(["pop", "classical", "pop", "pop", "pop", "jazz", "rock", "pop", "pop", "rock"][class_index])

1/1 [==============================] - 0s 109ms/step
[[0.04367118 0.04397912 0.00316085 0.0446478  0.0230275  0.05047956
  0.16053061 0.01051805 0.61577594 0.00420941]]
pop
1/1 [==============================] - 0s 15ms/step
[[0.25737303 0.15013072 0.09295966 0.04810232 0.02716121 0.19518131
  0.02369707 0.11211577 0.04032539 0.05295353]]
pop
1/1 [==============================] - 0s 21ms/step
[[3.1141825e-02 5.8526765e-03 7.6245185e-04 5.3298008e-03 1.4297360e-01
  2.8101832e-04 2.2176361e-02 3.1278498e-02 7.5666213e-01 3.5416703e-03]]
pop
pop


In [5]:
import librosa

# 加载音频文件
audio_file = './dataset/audio_format/blues/blues.00000.wav'  # 替换为您的音频文件路径
y, sr = librosa.load(audio_file)

# 1. 音高和旋律特征
chroma = librosa.feature.chroma_cqt(y=y, sr=sr)

# 2. 节奏和节拍
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
beat_times = librosa.frames_to_time(beat_frames, sr=sr)

# 3. 和声和和弦结构
harmonic = librosa.effects.harmonic(y)
harmonic_features = librosa.feature.chroma_cqt(y=harmonic, sr=sr)

# 4. 音色和乐器使用
mfccs = librosa.feature.mfcc(y=y, sr=sr)

# 5. 动态范围和强度
S, phase = librosa.magphase(librosa.stft(y))
rms = librosa.feature.rms(S=S)

# 6. 时间结构和形式
temporal_onset = librosa.onset.onset_strength(y=y, sr=sr)

# 7. 时域特征
zero_crossing_rate = librosa.feature.zero_crossing_rate(y)

# 输出每个特征的shape
print("Chroma (Pitch):", chroma.shape)
print("Tempo (Rhythm):", beat_times.shape)
print("Harmonic Features (Harmony):", harmonic_features.shape)
print("MFCCs (Timbre):", mfccs.shape)
print("RMS (Dynamics):", rms.shape)
print("Temporal Onset (Form):", temporal_onset.shape)
print("Zero Crossing Rate (Time-domain):", zero_crossing_rate.shape)

Chroma (Pitch): (12, 1293)
Tempo (Rhythm): (61,)
Harmonic Features (Harmony): (12, 1293)
MFCCs (Timbre): (20, 1293)
RMS (Dynamics): (1, 1293)
Temporal Onset (Form): (1293,)
Zero Crossing Rate (Time-domain): (1, 1293)
